In [ ]:
%%writefile /content/sap1_asm.py
#!/usr/bin/env python3
import re, sys

# Your ISA (operand_needed = True/False)
ISA = {
    "LDA":      (0x1, True),
    "LDB":      (0x2, True),
    "ADD":      (0x3, False),
    "COMPARE":  (0xD, False),
    "HLT":      (0xF, False),
}

def is_binary_token(s):
    return re.fullmatch(r"[01]{1,8}", s) is not None

def parse_number(tok):
    tok = tok.strip()
    # binary (e.g., 1010 or 00011000)
    if is_binary_token(tok):
        return int(tok, 2)
    # hex (e.g., 0x1A / 0X1A / A / F)
    if tok.startswith(("0x","0X")):
        return int(tok, 16)
    if len(tok)==1 and tok.upper() in "0123456789ABCDEF":
        return int(tok, 16)
    # decimal fallback
    return int(tok, 10)

def strip_comment(line: str) -> str:
    # remove anything after ; or #
    if ";" in line: line = line.split(";",1)[0]
    if "#" in line: line = line.split("#",1)[0]
    return line.strip()

def tokenize(src: str):
    for raw in src.splitlines():
        line = strip_comment(raw)
        if line:
            yield line

def first_pass(lines):
    """
    Builds label table + a list of parsed lines.
    PC wraps in lower 4 bits (0..15) for 16-byte memory.
    Supports:
      LABEL:         (optional) followed by instruction
      ORG addr       (set PC)
      DATA value     (write raw byte)
      <MNEMONIC> [operand]
    """
    pc = 0
    labels = {}
    parsed = []

    for line in lines:
        # label?
        if ":" in line:
            lab, rest = line.split(":", 1)
            lab = lab.strip()
            rest = rest.strip()
            if not lab:
                raise ValueError("Empty label definition.")
            if lab in labels:
                raise ValueError(f"Duplicate label: {lab}")
            labels[lab] = pc
            if not rest:
                # allow label-only line
                parsed.append(("NOPLINE", [], pc))
                continue
            line = rest

        toks = line.replace(",", " ").split()
        op = toks[0].upper()
        args = toks[1:]

        parsed.append((op, args, pc))

        if op == "ORG":
            if not args:
                raise ValueError("ORG needs an address.")
            pc = parse_number(args[0]) & 0xF
        else:
            pc = (pc + 1) & 0xF

    return labels, parsed

def assemble(src: str):
    mem = [0x00]*16
    labels, lines = first_pass(list(tokenize(src)))

    pc = 0
    for op, args, at in lines:
        if op == "NOPLINE":
            pc = at
            continue

        if op == "ORG":
            pc = parse_number(args[0]) & 0xF
            continue

        if op == "DATA":
            if not args:
                raise ValueError(f"DATA needs a value at address {pc:01X}")
            val = parse_number(args[0]) & 0xFF
            mem[pc] = val
            pc = (pc + 1) & 0xF
            continue

        if op not in ISA:
            raise ValueError(f"Unknown instruction: {op} at address {pc:01X}")

        opcode, needs_operand = ISA[op]
        operand = 0
        if needs_operand:
            if not args:
                raise ValueError(f"{op} needs an operand at address {pc:01X}")
            a = args[0]
            # label or number
            if a in labels:
                operand = labels[a] & 0xF
            else:
                operand = parse_number(a) & 0xF

        mem[pc] = ((opcode & 0xF) << 4) | (operand & 0xF)
        pc = (pc + 1) & 0xF

    return mem

def main():
    import argparse
    parser = argparse.ArgumentParser(description="SAP-1 assembler (LDA/LDB/ADD/COMPARE/HLT) -> 16-byte hex image")
    parser.add_argument("file", nargs="?", help="assembly source file (optional)")
    parser.add_argument("--list", action="store_true", help="print address-by-address listing")
    args, _ = parser.parse_known_args()

    if args.file:
        with open(args.file, "r", encoding="utf-8") as f:
            src = f.read()
    else:
        print("Enter assembly. Type END on a line by itself to finish:")
        buf=[]
        while True:
            try:
                line = input()
            except EOFError:
                break
            if line.strip() == "END":
                break
            buf.append(line)
        src = "\n".join(buf)

    mem = assemble(src)

    if args.list:
        print("Addr  Hex  Bin")
        print("--------------")
        for i,b in enumerate(mem):
            print(f"{i:>4}  {b:02X}  {b:08b}")
        print()
    print(" ".join(f"{b:02X}" for b in mem))

if __name__ == "__main__":
    main()



Overwriting /content/sap1_asm.py


In [ ]:
%%writefile /content/prog.asm
; Test program using LDA, LDB, ADD, COMPARE, HLT
ORG 0
LDA 8
LDB 9
ADD
COMPARE
HLT
; Data section
ORG 8
DATA 5
DATA 7


Overwriting /content/prog.asm


In [ ]:
!python /content/sap1_asm.py /content/prog.asm


18 29 30 D0 F0 00 00 00 05 07 00 00 00 00 00 00
